In [277]:
%pip install --upgrade pillow

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pillow in /Users/hemantic/Library/Python/3.8/lib/python/site-packages (7.0.0)
Note: you may need to restart the kernel to use updated packages.


In [279]:
from openpyxl import Workbook, load_workbook
import xml.etree.ElementTree as ET
import urllib.request
import json
import csv
import math
import re
from PIL import Image

In [218]:
# эти настройки надо менять
file_out = 'data/Обучающая игра для малышей.xlsx'
category_id = '6644'
file_website_parse_results = 'data/io.json'
file_prices_catalog = 'data/catalog_4_.csv'

# эти настройки менять НЕ НАДО
yamarket_url = 'https://игрушки-оптом-купить.рф/export/yamarket.xml'
yamarket_file_path = 'data/yamarket.xml'
start_row = 4

In [163]:
# загрузка шаблона XLSX
wb = load_workbook(filename=file_out)
ws = wb['Шаблон для поставщика']

# загрузка файла YML
tree = ET.parse(yamarket_file_path)
root = tree.getroot()

# загрузка данных с сайта
with open(file_website_parse_results) as f:
    website_parse_results = json.load(f)
    
# загрузка каталога с ценами
prices_catalog = {}

with open(file_prices_catalog, encoding="CP154") as csvfile:
    is_header = False
    header = []
    csv_reader = csv.reader(csvfile, delimiter='\t')
    
    for row_reader in csv_reader:
        if is_header is False:
            is_header = True
            header = row_reader
        else:
            row = {}
            index = 0

            for cell in row_reader:
                row[header[index]] = cell 
                index += 1

            prices_catalog[row['sku']] = row

In [275]:
'''one array to rule them all

В ключах массива – артикулы, данные расположены по ключам prices_catalog, yml, website
По каждому артикулу для каждого из источников данных записываются поля:

'type': {
    'name': None,
    'price': None,
    'weight': None,
    'dimensions_raw': None,
    'dimensions_calculated': None,
    'categories': None,
    'pictures': None,
}
'''

stocks = {}

# проходимся по yml
for offer in root.iter('offer'):
    name = None
    price = None
    weight = None
    dimensions_raw = None
    dimensions_calculated = None
    categories = None
    pictures = None
    
    if offer.find('name') is not None:
        name = str.capitalize(offer.find('name').text)
        
    if offer.find('price') is not None:
        price = float(offer.find('price').text)
    
    if offer.find('weight') is not None:
        weight = float(offer.find('weight').text) * 1000
        
    if offer.findall('categoryId') is not None:
        categories = list(map(lambda x: x.text, offer.findall('categoryId')))

    if offer.findall('picture') is not None:
        pictures = list(map(lambda x: x.text, offer.findall('picture')))
        
    # готовим размеры
    if name is not None:
        re_dimensions_1 = re.compile(r'[\d,]+\s*[x|х]\s*[\d,]+[x|х]*\s*[\d,]*\s*см')
        match_1 = re_dimensions_1.search(name)

        re_dimensions_2 = re.compile(r'[\d,]+\s*[x|х]\s*[\d,]+[x|х]*\s*[\d,]*\s*мм')
        match_2 = re_dimensions_2.search(name)

        if match_1 is not None:
            dimensions_raw = match_1.group()
            d_string_clean = dimensions_raw.replace(' ', '').replace('см', '')
            
            dimensions_calculated = sorted(list(map(lambda x: float(x.replace(',', '.')) * 10,  d_string_clean.split('х'))))
            
        if match_2 is not None:
            dimensions_raw = match_2.group()
            d_string_clean = dimensions_raw.replace(' ', '').replace('мм', '')
            
            dimensions_calculated = sorted(list(map(lambda x: float(x.replace(',', '.')),  d_string_clean.split('x'))))
        
    stocks[offer.find('vendorCode').text] = {
        'yml': {
            'name': name,
            'price': price,
            'weight': weight,
            'dimensions_raw': dimensions_raw,
            'dimensions_calculated': dimensions_calculated,
            'categories': categories,
            'pictures': pictures,
        }
    }

# проходимся по прайс-листу
for article, sku in prices_catalog.items():
    name = sku['name']
    price = float(sku['price'].replace(',', '.'))
    weight = float(sku['weight'].replace(',', '.')) * 1000
    dimensions_raw = None
    dimensions_calculated = None
    categories = sku['category']
    pictures = None
    
    # готовим размеры
    if sku['size'] is not None and sku['size'] != '0x0x0':
        dimensions_raw = sku['size']
        dimensions_calculated = sorted(list(map(lambda x: float(x.replace(',', '.')) * 10,  dimensions_raw.split('x'))), reverse=True)
    
    prices_catalog_sku = {
        'name': name,
        'price': price,
        'weight': weight,
        'dimensions_raw': dimensions_raw,
        'dimensions_calculated': dimensions_calculated,
        'categories': categories,
        'pictures': pictures,
    }
    
    if sku['sku'] in stocks:
        stocks[sku['sku']]['prices_catalog'] = prices_catalog_sku
    else:
        stocks[sku['sku']] = {
            'prices_catalog': prices_catalog_sku
        }

# проходимся по результатам парсинга сайта
for parsed_data in website_parse_results:
    name = None
    price = float(parsed_data['price'].replace(',', '.').replace(' ', ''))
    weight = float(parsed_data['weight'].replace(',', '.').replace(' ', '')) * 1000
    dimensions_raw = None
    dimensions_calculated = None
    categories = None
    pictures = None
    
    # готовим размеры
    if parsed_data['dimensions'] is not None:
        dimensions_raw = parsed_data['dimensions']
        dimensions_calculated = sorted(list(map(lambda x: float(x.replace(',', '.')) * 10,  dimensions_raw.split(' x '))), reverse=True)
    
    website_sku = {
        'name': name,
        'price': price,
        'weight': weight,
        'dimensions_raw': dimensions_raw,
        'dimensions_calculated': dimensions_calculated,
        'categories': categories,
        'pictures': pictures,
    }
    
    if parsed_data['id'] in stocks:
        stocks[parsed_data['id']]['website'] = website_sku
    else:
        stocks[parsed_data['id']] = {
            'website': website_sku
        }

In [259]:
# проводим проверки и статистику
def ppercent(smaller, bigger):
    return str(round(smaller / bigger * 100, 2)) + '%'

prices_catalog_count = 0
yml_count = 0
website_count = 0

stocks_len_count = len(stocks)

for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        prices_catalog_count += 1
    
    if 'yml' in sku:
        yml_count += 1
        
    if 'website' in sku:
        website_count += 1

print('Stocks:')
print(f'All: {stocks_len_count}')
print(f'Catalog: {prices_catalog_count} ({ppercent(prices_catalog_count, stocks_len_count)})')
print(f'Yml: {yml_count} ({ppercent(yml_count, stocks_len_count)})')
print(f'Website: {website_count} ({ppercent(website_count, stocks_len_count)})')
print()



dm_catalog_count = 0
dm_yml_count = 0
dm_website_count = 0

for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        if sku['prices_catalog']['dimensions_raw'] is not None:
            dm_catalog_count += 1
    
    if 'yml' in sku:
        if sku['yml']['dimensions_raw'] is not None:
            dm_yml_count += 1
        
    if 'website' in sku:
        if sku['website']['dimensions_raw'] is not None:
            dm_website_count += 1


print('Dimensions raw:')
print(f'In catalog: {dm_catalog_count}')
print(f'In yml: {dm_yml_count}')
print(f'In website: {dm_website_count}')
print()


dmc_catalog_count = 0
dmc_yml_count = 0
dmc_website_count = 0

for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        if sku['prices_catalog']['dimensions_calculated'] is not None and len(sku['prices_catalog']['dimensions_calculated']) > 1:
            dmc_catalog_count += 1
    
    if 'yml' in sku:
        if sku['yml']['dimensions_calculated'] is not None and len(sku['yml']['dimensions_calculated']) > 1:
            dmc_yml_count += 1
        
    if 'website' in sku:
        if sku['website']['dimensions_calculated'] is not None and len(sku['website']['dimensions_calculated']) > 1:
            dmc_website_count += 1


print('Dimensions calculated:')
print(f'In catalog: {dmc_catalog_count}')
print(f'In yml: {dmc_yml_count}')
print(f'In website: {dmc_website_count}')
print()

weight_catalog_count = 0
weight_yml_count = 0
weight_website_count = 0

for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        if sku['prices_catalog']['weight'] is not None and sku['prices_catalog']['weight'] > 0:
            weight_catalog_count += 1
    
    if 'yml' in sku:
        if sku['yml']['weight'] is not None and sku['yml']['weight'] > 0:
            weight_yml_count += 1
        
    if 'website' in sku:
        if sku['website']['weight'] is not None and sku['website']['weight'] > 0:
            weight_website_count += 1


print('Weight:')
print(f'In catalog: {weight_catalog_count} ({ppercent(weight_catalog_count, stocks_len_count)})')
print(f'In yml: {weight_yml_count} ({ppercent(weight_yml_count, stocks_len_count)})')
print(f'In website: {weight_website_count} ({ppercent(weight_website_count, stocks_len_count)})')
print()

pictures_catalog_count = 0
pictures_yml_count = 0
pictures_website_count = 0

for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        if sku['prices_catalog']['pictures'] is not None and len(sku['prices_catalog']['pictures']) > 0:
            pictures_catalog_count += 1
    
    if 'yml' in sku:
        if sku['yml']['pictures'] is not None and len(sku['yml']['pictures']) > 0:
            pictures_yml_count += 1
        
    if 'website' in sku:
        if sku['website']['pictures'] is not None and len(sku['website']['pictures']) > 0:
            pictures_website_count += 1


print('Pictures:')
print(f'In catalog: {pictures_catalog_count} ({ppercent(pictures_catalog_count, stocks_len_count)})')
print(f'In yml: {pictures_yml_count} ({ppercent(pictures_yml_count, stocks_len_count)})')
print(f'In website: {pictures_website_count} ({ppercent(pictures_website_count, stocks_len_count)})')
print()

Stocks:
All: 4870
Catalog: 4764 (97.82%)
Yml: 4752 (97.58%)
Website: 4444 (91.25%)

Dimensions raw:
In catalog: 4387
In yml: 277
In website: 86

Dimensions calculated:
In catalog: 4387
In yml: 277
In website: 86

Weight:
In catalog: 4357 (89.47%)
In yml: 4346 (89.24%)
In website: 62 (1.27%)

Pictures:
In catalog: 0 (0.0%)
In yml: 4715 (96.82%)
In website: 0 (0.0%)



In [260]:
for article, sku in stocks.items():
    if 'prices_catalog' in sku:
        #print(f'{sku["prices_catalog"]["dimensions_raw"]} => {sku["prices_catalog"]["dimensions_calculated"]} ({sku["prices_catalog"]["name"]})')
        if sku["prices_catalog"]["dimensions_raw"] is None:
            pass
            #print(f'{sku["prices_catalog"]["name"]} / {stocks[article]["yml"]["dimensions_calculated"]}')

In [223]:
# смотрим, какие поля есть в Озоновском шаблоне и делаем заготовку для конфига
# заодно кешируем названия полей
template_headers = {}
print('param_template = {')

for col in range(1, 100):
    cell = ws.cell(column=col, row=3)
    cell_value = cell.value
    
    if cell_value is None:
        break
        
    template_headers[cell.value] = cell.column_letter
    
    print(f'    "{cell_value}": None,  # {cell.column_letter}')
    
print('}')

param_template = {
    "№": None,  # A
    "Артикул*": None,  # B
    "Название товара": None,  # C
    "Цена, руб.*": None,  # D
    "Цена до скидки, руб.": None,  # E
    "Цена с Ozon Premium, руб.": None,  # F
    "НДС, %*": None,  # G
    "Ozon ID": None,  # H
    "Коммерческий тип*": None,  # I
    "Штрихкод (Серийный номер / EAN)": None,  # J
    "Бренд": None,  # K
    "Подробное описание товара": None,  # L
    "Вес в упаковке, г*": None,  # M
    "Ширина упаковки, мм*": None,  # N
    "Высота упаковки, мм*": None,  # O
    "Длина упаковки, мм*": None,  # P
    "Ссылка на главное фото*": None,  # Q
    "Ссылки на дополнительные фото": None,  # R
    "Ссылки на фото 360": None,  # S
    "Ссылки на фото аннотаций": None,  # T
    "Артикул фото": None,  # U
    "Тип*": None,  # V
    "Страна-изготовитель": None,  # W
    "Пол": None,  # X
    "Образец цвета": None,  # Y
    "Вес товара, г": None,  # Z
    "Методики обучения": None,  # AA
    "Советы маме": None,  # AB
    "Развити

In [224]:
param_template = {
    "Коммерческий тип*": 'Обучающая игра для малышей',  # I
    "Тип*": 'Обучающая игра',  # V
    "Страна-изготовитель": 'Россия',  # W
    "Развитие навыков": 'Моторика',  # AC
    "Материал": 'Дерево',  # AP
    "Товар бывший в употреблении": 'Нет',  # AQ
    "Возрастной диапазон": 'от 1 до 3 лет',  # AS
    "Возраст ребенка": 'От 1 года',  # AU
    "Вид детской игры": 'Развивающие',  # AV
}

In [225]:
# определяем список категорий по одной родительской
#category_ids = []
#for cat in root.iter('category'):
#    if cat.get('parentId') == '5929':
#        category_ids.append(cat.get('id'))

In [276]:
# ищем позиции заданных категорий
current_row = start_row

def decide_bulk_price(sku):
    if 'prices_catalog' in sku and sku['prices_catalog']['price'] is not None:
        return sku['prices_catalog']['price']
    
    if 'yml' in sku and sku['yml']['price'] is not None:
        return sku['yml']['price']
    
    return 0

def decide_weight(sku):
    if 'prices_catalog' in sku and sku['prices_catalog']['weight'] is not None and sku['prices_catalog']['weight'] != 0:
        return sku['prices_catalog']['weight']
    
    if 'yml' in sku and sku['yml']['weight'] is not None and sku['yml']['weight'] != 0:
        return sku['yml']['weight']
    
    if 'website' in sku and sku['website']['weight'] is not None and sku['website']['weight'] != 0:
        return sku['website']['weight']
    
    return 500

def decide_dimensions(sku):
    dimensions = []
    
    if 'prices_catalog' in sku and sku['prices_catalog']['dimensions_calculated'] is not None:
        dimensions = sku['prices_catalog']['dimensions_calculated']
    
    if 'yml' in sku and sku['yml']['dimensions_calculated'] is not None:
        dimensions = sku['yml']['dimensions_calculated']
    
    if 'website' in sku and sku['website']['dimensions_calculated'] is not None:
        dimensions = sku['website']['dimensions_calculated']
    
    if len(dimensions) < 3:
        for _ in range(0, 3-len(dimensions)):
            dimensions.append(0)
            
    return dimensions

def decide_pictures(sku):
    if 'yml' in sku and sku['yml']['pictures'] is not None:
        return sku['yml']['pictures']
    
    return []

def get_main_picture(pictures):
    if len(pictures):
        return pictures[0]
    
    return None

def get_other_pictures(pictures):
    if len(pictures):
        pictures.pop(0)
        return ' '.join(map(lambda x : x.replace('http://', ''), pictures))
    
    return None

for article, sku in stocks.items():
    if 'yml' in sku and category_id in sku['yml']['categories']:
        article = article
        article_name = sku['yml']['name']
        price_bulk = decide_bulk_price(sku)
        dimensions = decide_dimensions(sku)
        weight = decide_weight(sku)
        pictures = decide_pictures(sku)
        picture_main = get_main_picture(pictures)  # pictures.pop(0).text
        picture_other = get_other_pictures(pictures)  # ' '.join(map(lambda x : x.text.replace('http://', ''), pictures))
        
        price = math.ceil(price_bulk * 0.8 * 4 / 10) * 10
        price_with_discount = price * 0.7
        
        row_vars = {
            "Артикул*": article,  # B
            "Название товара": article_name,  # C
            "Цена, руб.*": price_with_discount,  # D
            "Цена до скидки, руб.": price,  # E
            "НДС, %*": 20,  # G
            "Вес в упаковке, г*": weight,  # M
            "Ссылка на главное фото*": picture_main,  # Q
            "Ссылки на дополнительные фото": picture_other,  # R
            "Длина упаковки, мм*": dimensions[0],  # P
            "Ширина упаковки, мм*": dimensions[1],  # N
            "Высота упаковки, мм*": dimensions[2],  # O
        }
            
        row_data = {**param_template, **row_vars}
        
        for field, value in row_data.items():
            column_letter = template_headers[field]
            ws[f'{column_letter}{current_row}'] = value
        
        print(f'row {current_row} processed')
        
        current_row += 1

wb.save(file_out)

row 4 processed
row 5 processed
row 6 processed
row 7 processed
row 8 processed
row 9 processed
row 10 processed
row 11 processed
row 12 processed
row 13 processed
row 14 processed
row 15 processed
row 16 processed
row 17 processed
row 18 processed
row 19 processed
row 20 processed
row 21 processed
row 22 processed
row 23 processed
row 24 processed
row 25 processed
row 26 processed
row 27 processed
row 28 processed
row 29 processed
row 30 processed
row 31 processed
row 32 processed
row 33 processed
row 34 processed
row 35 processed
row 36 processed
row 37 processed
row 38 processed
row 39 processed
row 40 processed


In [286]:
image = Image.open('data/44474_big.png')
print(image.size)

resized = image.resize(list(map(lambda x: x * 4,image.size)), Image.ANTIALIAS)

image.show()
resized.show()

(137, 88)
